# Sistema de Recomendação de Músicas em Tempo Real

## Ferramentas utilizadas:

- Api Spotify
- Machine Leraning
- PySpark
- Spark Streaming
- Apache Kafka

## Aqruivo Producer do Kafka

In [1]:
# Versão do Python que estou utlizando neste projeto
from platform import python_version
print(python_version())

3.9.13


In [2]:
# Fazendo a instalação do conector entre o kafka e o python
# https://kafka-python.readthedocs.io/en/master/
!pip install -q kafka-python

In [3]:
# Imports
import time
import random
import kafka
import numpy as np
import pandas as pd
import json as dumps
from kafka import KafkaProducer
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Endereço do servidor kafka
SERVER = 'localhost:9092'

In [5]:
# Nome do Tópico
TOPIC = 'dsaminiprojeto7'

In [6]:
# Carregando o conjunto de dados de músicas
df_projeto = pd.read_csv('dados/dataset.csv')

In [7]:
# Ajustando o formato de 3 colunas importantes: order_id, Artist Name(s) e Artist IDs
df_projeto['order_id'] = np.arange(len(df_projeto))
df_projeto['Artist Name(s)'] = df_projeto['Artist Name(s)'].str.replace('[^a-zA-Z]', '')
df_projeto['Artist IDs'] = df_projeto['Artist IDs'].str.replace('[^a-zA-z]', '')

In [8]:
df_projeto.shape

(4399, 24)

In [9]:
df_projeto.head(10)

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,order_id
0,22a0Ji6EQKkY0tBohlN4Od,qLyYYhSlsjwymwVKwW,There You Are,There You Are,KirstenLudwig,2018-08-06,231240,2,spotify:user:predict0,2018-08-28T19:51:58Z,...,-5.596,0,0.0304,0.334000,0.282000,0.1050,0.316,129.856,4,0
1,4J39ZEbwqHwtWLImUKmrn9,CRfAxYjJsDBHwvWFnjaRRRPXwFwQmoTNqNHBGU,88 Days,Heat,SaraKingIanOlney,2018-08-04,227961,8,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.749,1,0.0333,0.134000,0.582000,0.1340,0.233,155.062,4,1
2,0a12d4HUjOmQSqHqLopWYx,hytHTGTflktWAhKcxQ,Castaway,Castaway,ARZLEE,2018-08-10,230000,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.290,1,0.0314,0.110000,0.000032,0.1190,0.290,83.988,4,2
3,4u1DykFW1HjYAGNoDCiXfC,WjyoJHRHlTbUTZTwqpAgeqmtJlARXjon,Arouse,Arouse,Shagabondgoodboynoah,2018-08-03,213913,30,spotify:user:predict0,2018-08-28T19:51:58Z,...,-6.066,1,0.4330,0.072800,0.000000,0.3680,0.533,91.961,4,3
4,0u7JZm9ORerlZnnxxSdMwl,AdKmjgFzpcTvmVfGwR,Lonely,Lonely,Hayleau,2018-08-10,258738,21,spotify:user:predict0,2018-08-28T19:51:58Z,...,-3.921,0,0.0406,0.016900,0.000630,0.0542,0.577,98.954,4,4
5,0wuy2BYIVLbflFDqnR9Jay,kCwrYUFSJCubbbnZrE,Orsay,Strange Affairs,TheSvens,2018-08-03,413658,6,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.858,1,0.0316,0.048600,0.886000,0.1280,0.283,122.992,4,5
6,6LkIZZRrPQIbHMyBR5mTc2,TOsWuafqeWtrvYXqbnYAV,Nurture,Comrade,IslandFox,2018-08-09,191641,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.415,0,0.0504,0.015900,0.639000,0.1810,0.266,133.925,4,6
7,5U27fxNSd27XtX876xUsfV,HsKUExgNcRJojPmBcNqzgwpvzedAIjuDBM,Dinosaur Hair - Remix,Dinosaur Hair,AndyFerroCharlieConway,2018-08-10,257152,1,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.086,1,0.0383,0.456000,0.920000,0.1350,0.663,125.908,4,7
8,5ogJOpmyDsvrAdttU6JLnN,gslbnQQLLcNzfjnxQY,Breathing Underwater,Long Way Down,MorningWars,2018-08-03,174999,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.258,0,0.0461,0.000005,0.020400,0.1150,0.477,150.042,4,8
9,65rLHt6A58MFRxlNWVDU1Z,WlYiRrlrChWktQDo,Summer,Summer,NoSo,2018-08-01,232746,22,spotify:user:predict0,2018-08-28T19:51:58Z,...,-7.517,1,0.0380,0.016800,0.007230,0.0706,0.210,123.962,4,9


In [10]:
# Convertendo o dataframe em um dicionário de músicas
# O kafka armazena os dados em formato de chave e valor, por isso temos que transformar o dataset em dicionário
dict_musicas = df_projeto.to_dict(orient = 'records')

In [11]:
dict_musicas[1:3]

[{'Spotify ID': '4J39ZEbwqHwtWLImUKmrn9',
  'Artist IDs': 'CRfAxYjJsDBHwvWFnjaRRRPXwFwQmoTNqNHBGU',
  'Track Name': '88 Days',
  'Album Name': 'Heat',
  'Artist Name(s)': 'SaraKingIanOlney',
  'Release Date': '2018-08-04',
  'Duration (ms)': 227961,
  'Popularity': 8,
  'Added By': 'spotify:user:predict0',
  'Added At': '2018-08-28T19:51:58Z',
  'Genres': 'bedroom pop',
  'Danceability': 0.335,
  'Energy': 0.401,
  'Key': 3,
  'Loudness': -10.749,
  'Mode': 1,
  'Speechiness': 0.0333,
  'Acousticness': 0.134,
  'Instrumentalness': 0.582,
  'Liveness': 0.134,
  'Valence': 0.233,
  'Tempo': 155.062,
  'Time Signature': 4,
  'order_id': 1},
 {'Spotify ID': '0a12d4HUjOmQSqHqLopWYx',
  'Artist IDs': 'hytHTGTflktWAhKcxQ',
  'Track Name': 'Castaway',
  'Album Name': 'Castaway',
  'Artist Name(s)': 'ARZLEE',
  'Release Date': '2018-08-10',
  'Duration (ms)': 230000,
  'Popularity': 0,
  'Added By': 'spotify:user:predict0',
  'Added At': '2018-08-28T19:51:58Z',
  'Genres': nan,
  'Danceability'

In [ ]:
# Construindo o Kafka Producer
if __name__ == '__main__':
    
    # Cria o producer
    producer = KafkaProducer(bootstrap_servers = SERVER, # nome do servidor
                             value_serializer = lambda x: x.encode('utf-8')) # serializador para evitar problemas com acentos, etc
    
    # Variáveis de controle
    send = []
    send = None
    
    # Loop pelo dicionário de músicas
    for musica in dict_musicas:
        
        # Cria a lista com dados que serão enviados para o kafka
        sending = []
        
        # Append de cada coluna que queremos enviar ao kafka (não serão todas)
        sending.append(musica['order_id'])
        sending.append(musica['Spotify ID'])
        sending.append(musica['Track Name'])
        sending.append(musica['Popularity'])
        sending.append(musica['Duration (ms)'])
        sending.append(musica['Artist Name(s)'])
        sending.append(musica['Release Date'])
        sending.append(musica['Danceability'])
        sending.append(musica['Energy'])
        sending.append(musica['Key'])
        sending.append(musica['Loudness'])
        sending.append(musica['Mode'])
        sending.append(musica['Speechiness'])
        sending.append(musica['Acousticness'])
        sending.append(musica['Instrumentalness'])
        sending.append(musica['Liveness'])
        sending.append(musica['Valence'])
        sending.append(musica['Tempo'])
        sending.append(musica['Time Signature'])
        
        # Junta tudo e transforma em json
        musica = ','.join(str(v) for v in sending)
        
        # Envia os dados para o tópico
        print('Pŕoxima música:' )
        print(musica)
        producer.send(TOPIC, musica)
        time.sleep(1) # é importante para dar um espaçamento, evita possíveis erros
        
    print('Concluído')
                              

Pŕoxima música:
0,22a0Ji6EQKkY0tBohlN4Od,There You Are,2,231240,KirstenLudwig,2018-08-06,0.487,0.707,9,-5.596,0,0.0304,0.334,0.282,0.105,0.316,129.856,4
Pŕoxima música:
1,4J39ZEbwqHwtWLImUKmrn9,88 Days,8,227961,SaraKingIanOlney,2018-08-04,0.335,0.401,3,-10.749,1,0.0333,0.134,0.582,0.134,0.233,155.062,4
Pŕoxima música:
2,0a12d4HUjOmQSqHqLopWYx,Castaway,0,230000,ARZLEE,2018-08-10,0.553,0.422,1,-11.29,1,0.0314,0.11,3.25e-05,0.119,0.29,83.988,4
Pŕoxima música:
3,4u1DykFW1HjYAGNoDCiXfC,Arouse,30,213913,Shagabondgoodboynoah,2018-08-03,0.67,0.751,1,-6.066,1,0.433,0.0728,0.0,0.368,0.533,91.961,4
Pŕoxima música:
4,0u7JZm9ORerlZnnxxSdMwl,Lonely,21,258738,Hayleau,2018-08-10,0.67,0.709,8,-3.921,0,0.0406,0.0169,0.00063,0.0542,0.577,98.954,4
Pŕoxima música:
5,0wuy2BYIVLbflFDqnR9Jay,Orsay,6,413658,TheSvens,2018-08-03,0.61,0.444,0,-11.858,1,0.0316,0.0486,0.886,0.128,0.283,122.992,4
Pŕoxima música:
6,6LkIZZRrPQIbHMyBR5mTc2,Nurture,0,191641,IslandFox,2018-08-09,0.324,0.808,7,-11.415,0,0.0504,0.0159,0.63

Pŕoxima música:
55,371vbxMzehOmYiMdGecWyx,Focus,8,273573,Blubird,2018-07-20,0.469,0.574,9,-7.636,0,0.0334,0.635,1.99e-06,0.0749,0.179,75.994,4
Pŕoxima música:
56,55SqDXuhCPXqZTTGGRDUTR,Multitudes so Long,23,272000,Shortwaved,2018-07-19,0.592,0.116,0,-21.08,1,0.0331,0.718,0.867,0.124,0.23,74.999,4
Pŕoxima música:
57,4BwliC8tLpXWygquXnLpvz,Downtown & Cigarettes,40,109871,Bbyafricka,2018-07-20,0.771,0.329,0,-12.227,0,0.0382,0.634,0.0,0.112,0.444,111.09,4
Pŕoxima música:
58,2giwdykXaXqHEiW4G8PiSE,Distance,0,146041,KayoGenesis,2018-07-27,0.715,0.605,0,-6.619,1,0.0531,0.283,0.0,0.0943,0.821,85.078,4
Pŕoxima música:
59,7Jv6xrIAqdGLw3F24ZDIBT,Loso,8,151570,NinaTech,2018-07-16,0.732,0.451,1,-10.742,1,0.0378,0.127,0.00183,0.134,0.115,140.002,4
Pŕoxima música:
60,2xSvltSNjuVS9G87v2Iqxr,Turbo Sideman,7,298560,Itoa,2018-07-20,0.798,0.791,2,-5.661,1,0.0637,0.000851,0.0443,0.0788,0.36,160.038,4
Pŕoxima música:
61,2WRR6NNGAuoMYqc8UtPLid,Black Noise,0,210391,DJKhalabTeneshaTheWordsmith,2018-07-20,0.872

Pŕoxima música:
110,6sVe6OJujha3anyxixneit,Immigrant Sons (Pesos & Gas),12,260223,GAIKA,2018-07-11,0.736,0.663,1,-7.194,0,0.0408,0.26,1.97e-05,0.0639,0.486,95.044,4
Pŕoxima música:
111,1cQCdozRmaErXowT6iDQML,TROJAN HORSE,65,266393,SEBASTIANPAUL,2018-07-20,0.651,0.726,8,-7.583,1,0.312,0.576,0.0152,0.272,0.657,89.305,4
Pŕoxima música:
112,2VkIb2YCSa4ovjvo54ufEl,FIRE DANCE,17,306115,CloseCounters,2018-02-08,0.781,0.703,5,-10.743,1,0.0641,0.00228,0.783,0.199,0.197,115.993,4
Pŕoxima música:
113,21V3vTt5YxudQ7JL7O8RHT,Big Age,0,213813,Toxe,2018-06-22,0.651,0.813,2,-6.737,0,0.0439,0.626,0.77,0.0821,0.549,160.053,4
Pŕoxima música:
114,2W6QwWallFIKsEHsI0GHb9,Alight,0,195469,TokyoProse,2018-07-20,0.446,0.0416,6,-20.786,1,0.0472,0.738,0.773,0.145,0.0395,116.424,5
Pŕoxima música:
115,09jCT85eBB11e1BgM2rzES,Venice,0,182729,extremelybadman,2018-07-13,0.53,0.663,8,-6.697,1,0.0579,0.413,0.00648,0.679,0.631,73.63,4
Pŕoxima música:
116,3EkDfIazeRkOUk2W4G9Cr1,Rosemvry,8,201600,MADANII,2018-06-08,0.826,0.

Pŕoxima música:
164,7eRbgI2QBucpMCd18xlzlV,Home Team,0,203486,IndigoDeSouza,2018-06-08,0.359,0.416,7,-10.319,1,0.0386,0.046,0.0286,0.119,0.0921,106.361,4
Pŕoxima música:
165,4cSZn3SShSy9FAKjt1UXfO,Isle Of Pines,13,205954,GOSH,2018-06-05,0.702,0.627,1,-5.654,0,0.0257,0.237,5.32e-05,0.115,0.591,114.969,4
Pŕoxima música:
166,60BVo7Q8xXpO6xyNfDe4sc,Silence,0,278060,DRYXPerryDavis,2018-06-15,0.616,0.248,11,-16.999,1,0.0301,0.0614,0.000178,0.138,0.508,137.961,4
Pŕoxima música:
167,5viCNvmAdb1qF0qmjHwYmF,Throat,7,194272,dreamcastmoe,2018-06-10,0.642,0.354,1,-12.876,0,0.0455,0.538,0.0989,0.108,0.17,123.993,4
Pŕoxima música:
168,4Y8ShhlBQDuA5AK5o5xvoC,Higher Vibration,4,167225,ClydeGuevara,2018-06-06,0.751,0.573,1,-6.458,1,0.285,0.198,0.0,0.108,0.237,77.469,4
Pŕoxima música:
169,4jeVou304TJ27MNsL9NaA0,lil bit,3,155541,happytreelatelate,2018-06-14,0.704,0.597,6,-10.605,0,0.281,0.389,0.0,0.0741,0.585,154.321,4
Pŕoxima música:
170,6YwiEI5fi4MPY01qEB0ZHb,Gaia,0,178733,BenBadaBoom,2018-06-11,0.613,0

Pŕoxima música:
218,525rpMqrjG91eBKDF0Rl22,Don't Fade Away,0,254061,KidRose,2018-06-01,0.68,0.67,5,-10.69,0,0.0415,0.0273,0.0977,0.18,0.144,129.997,4
Pŕoxima música:
219,2SjU23fW1G2TLA4jZAn2Sv,Rack$,11,108184,Kristhepirit,2018-05-30,0.918,0.531,1,-9.383,1,0.221,0.106,0.0,0.0967,0.54,120.017,4
Pŕoxima música:
220,1R1a2PejLhxfbFcAUMUOej,Unknown for Me,7,200000,KitMoroMina,2018-06-01,0.415,0.31,2,-12.338,1,0.049,0.293,0.00414,0.101,0.0488,95.975,4
Pŕoxima música:
221,1mGppNewxmuC9tpXhgZ2kO,Ocean,41,249660,dontask,2018-06-03,0.647,0.29,4,-16.196,1,0.0811,0.682,0.0,0.0965,0.271,74.987,4
Pŕoxima música:
222,23FL2h4RqCkbJAitknYPlx,Biggest Fool in the World,11,212309,FRITZ,2018-06-01,0.408,0.948,6,-4.757,1,0.0581,0.00163,2.71e-05,0.177,0.516,89.908,4
Pŕoxima música:
223,507FAuWKX1ApAWL7JmKupF,Neverland,7,204521,FERS,2018-06-08,0.653,0.702,2,-6.779,1,0.0509,0.211,0.183,0.116,0.327,115.001,4
Pŕoxima música:
224,3dub1a8fuCYOQcvKgBceHc,Snacks,0,270500,OscarLang,2018-05-31,0.605,0.478,10,-7.946,0,0

...

Pŕoxima música:
2861,3G9CeldVur6AWIYTCpBEKy,Nuh Nuh,4,342500,Joasihno,2016-05-06,0.52,0.542,5,-6.62,1,0.0285,0.984,0.144,0.102,0.213,96.986,4
Pŕoxima música:
2862,6usTUo1VZET3DLmiogVblo,Call You Back,3,246000,SlowShudder,2016-05-07,0.787,0.425,0,-7.953,1,0.0449,0.307,0.00281,0.0887,0.199,128.019,4
Pŕoxima música:
2863,7EnV4WLUWG8VHRCbWU6Aj7,Give To Receive,0,204721,RachelFoxx,2016-11-09,0.405,0.683,3,-8.903,1,0.2,0.0627,0.017,0.0948,0.108,97.104,4
Pŕoxima música:
2864,4J9T9NNdWSpSJcddyoJq43,100% (Below Zero),0,235355,GJnrKatVinter,2016-05-20,0.666,0.471,0,-6.736,0,0.152,0.372,1.94e-05,0.0982,0.283,103.653,4
Pŕoxima música:
2865,53fuvPRCiY6GueokPRkFG2,Kiki,0,226577,BATTS,2016-05-16,0.353,0.418,5,-11.182,0,0.0528,0.0604,0.0579,0.973,0.036,124.572,4
Pŕoxima música:
2866,4xKnLGhdyDU8dBddhLfFID,Over New York,0,288000,MindBath,2016-06-03,0.359,0.28,9,-13.79,0,0.0434,0.95,0.67,0.111,0.0716,79.703,4
Pŕoxima música:
2867,2JzBo8j1PZIzyxVcJqr7nO,Cinnamon,33,204000,Jome,2016-05-19,0.451,0.474,8,-1

Pŕoxima música:
2915,3O4Shtga9KLJgwDsfscX78,Back to You,6,211212,KYLI,2016-04-29,0.564,0.719,10,-9.64,0,0.288,0.0222,0.00385,0.0928,0.109,150.039,4
Pŕoxima música:
2916,6n8mh0WrdaeyRi8v7xHBrr,Like This,8,207600,Zipel,2016-04-29,0.774,0.637,2,-8.0,1,0.106,0.0424,0.002,0.0896,0.575,172.042,4
Pŕoxima música:
2917,5xshYxxYoLBsOmNOW83Uzj,Hits Me Hard,0,162329,JonLawless,2015-04-01,0.9,0.458,10,-10.941,1,0.211,0.237,0.0222,0.0884,0.784,109.419,4
Pŕoxima música:
2918,3eFQBRRILbmD7idR9f2RhY,Frequencies,13,217500,JAERA,2016-04-29,0.437,0.668,7,-7.806,0,0.0599,0.00469,0.000145,0.252,0.408,132.977,5
Pŕoxima música:
2919,4vEr51YAdwkD8Mz2StPLm4,Perk Up,0,228630,Ashdown,2016-04-28,0.705,0.684,5,-7.392,0,0.0572,0.183,3.39e-05,0.0702,0.771,94.992,4
Pŕoxima música:
2920,4akw05DOVu9zRKWW0TI3zO,Bones,0,222505,Ever,2016-04-22,0.59,0.63,7,-10.861,1,0.0507,0.174,0.0112,0.0943,0.0671,119.837,4
Pŕoxima música:
2921,3pS5SUvGmrzZUYjsWo7ISz,Back and Forth,9,211500,HolyOker,2016-05-05,0.882,0.329,5,-18.496,0,0.12

Pŕoxima música:
2968,2GKfkhelwv2bXbkqdsKUIG,Sixteen,7,265421,BRNFKDCamilleMichelleGray,2016-04-22,0.405,0.405,9,-9.321,0,0.0352,0.54,0.000163,0.172,0.123,109.633,4
Pŕoxima música:
2969,1xqxRj7WF8pHdEkTVewHQa,For What It's Worth,21,221357,NjomzaFKist,2016-04-15,0.612,0.484,8,-7.406,1,0.39,0.279,0.0,0.153,0.509,139.929,4
Pŕoxima música:
2970,6gLFGWwrTXTLXWQLuWz38x,Next Dance,24,279040,ManuelDarquart,2016-04-15,0.868,0.407,11,-7.977,0,0.0913,0.0181,0.727,0.113,0.736,118.019,4
Pŕoxima música:
2971,2GAGvST4mbygRgr5F4mRqf,Wake Up,4,262166,Blbird,2016-04-12,0.782,0.705,7,-9.595,0,0.038,0.000684,0.723,0.928,0.19,120.03,4
Pŕoxima música:
2972,32yfHSFeOicKSEPVyKAknC,Rehearsed Engineering,4,183072,KimBrown,2016-04-22,0.59,0.496,7,-14.497,1,0.0485,0.891,0.951,0.0866,0.048,109.009,4
Pŕoxima música:
2973,4dZhA5VGKpOsYZLRhXEcyw,Dead Heat,0,275251,Raime,2016-04-20,0.662,0.266,9,-15.485,1,0.0366,0.0227,0.847,0.115,0.06,122.017,4
Pŕoxima música:
2974,5MJoYAxRzNqzV1vWE8Bbse,Throne Keen,0,292097,DJOctopus

Pŕoxima música:
3021,5dHEwYmYHRKS7rRNGG7K9M,Hallelujah,0,156026,Mouthe,2016-04-08,0.751,0.595,7,-7.692,1,0.339,0.17,6.51e-05,0.176,0.664,100.023,4
Pŕoxima música:
3022,479xhIMpUhIw1cjUXe6OO4,Warm Magic,40,187826,JaduHeart,2016-04-07,0.71,0.366,0,-12.68,1,0.148,0.677,0.0786,0.582,0.321,115.068,4
Pŕoxima música:
3023,4kBrbBGK0vLW1CihB20Qtq,100K,0,203960,Antwon,2016-04-01,0.556,0.449,7,-10.363,1,0.227,0.0807,0.0372,0.446,0.484,89.984,4
Pŕoxima música:
3024,4o5YfvmPARvNODcPaW5vHL,The Cool Kids - Original Version,0,171000,BVRGER,2016-03-24,0.828,0.692,9,-7.658,0,0.0778,0.179,5.71e-06,0.175,0.682,120.052,4
Pŕoxima música:
3025,3ge1hYAnagOav3TcN7FUOy,Know You Better,0,209523,Mileo,2016-04-08,0.447,0.523,0,-6.474,1,0.165,0.193,0.0,0.296,0.45,84.148,5
Pŕoxima música:
3026,7aJcEXPSJ3xzeN9WOnhnzE,I,21,216784,vxDylanBrady,2016-04-01,0.428,0.338,8,-11.425,0,0.0564,0.523,0.0,0.0738,0.321,127.348,4
Pŕoxima música:
3027,71ymx3rPtyWwjehQ1j5ZGp,Our Best Reflections,0,297593,GoCozy,2016-04-08,0.477,0.591

Pŕoxima música:
3076,0MHKWsNYCVK1MbYS0LpFBF,DayLight,4,238000,SadeSerena,2016-03-25,0.697,0.743,6,-5.411,0,0.0392,0.0512,0.0,0.275,0.762,139.966,4
Pŕoxima música:
3077,1wN4hWLgIpvLCjJSMvwAjj,Used 2 This,2,192719,BadWlfy,2016-03-31,0.758,0.698,9,-7.632,0,0.0767,0.0309,0.00724,0.119,0.391,123.955,4
Pŕoxima música:
3078,0Z5v6Ng5AhScYxZuiiWEC7,Raindrops,0,232500,APOGEE,2016-03-23,0.51,0.791,5,-6.759,1,0.0395,0.000275,3.72e-05,0.106,0.518,159.892,4
Pŕoxima música:
3079,0VRFGCsvap5uy7OyDhRl30,The Other Me,0,266844,Laila,2016-03-29,0.693,0.656,1,-10.62,1,0.0419,0.1,0.38,0.0901,0.0972,108.023,4
